# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
data2=data.groupby(['CustomerID','ProductName']).sum('Quantity')
data2

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
pivot=pd.pivot_table(
    data2,
    values=['Quantity'],
    columns=['CustomerID'],
    index=['ProductName'],
    aggfunc={'sum'}).fillna(0)
pivot

Quantity                                \
                                       sum                                 
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp      0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Sausage Rolls              0.0   0.0   0.0   0.0   0.0   0.0   
Apricots - Dried                       1.0   0.0   0.0   0.0   1.0   0.0   
...                                    ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                   0.0   0.0   0.0   0.0   0.0   0.0   
Yoghurt Tubes                          0.0   0.0   0.0   0.0   0.0   0.0   
Yogurt - Blueberry, 175 Gr             0.0   1.0   0.0   0.0   0.0   0.0   
Yogurt - French Vanilla                1.0   0.0   0.0   1.0   0.0   0.0   
Zucchini - Yellow                      0.0   0.0   0.0   0.0   0.0   0.0   

                                                           ...              \
                                                           ...               
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube           0.0   1.0   0.0   0.0  ...   0.0  25.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0  ...  25.0  25.0   
Appetizer - Mushroom Tart           0.0   1.0   0.0   0.0  ...  25.0   0.0   
Appetizer - Sausage Rolls           0.0   0.0   0.0   0.0  ...   0.0   0.0   
Apricots - Dried                    0.0   0.0   0.0   0.0  ...   0.0  25.0   
...                                 ...   ...   ...   ...  ...   ...   ...   
Yeast Dry - Fermipan                0.0   0.0   0.0   0.0  ...   0.0   0.0   
Yoghurt Tubes                       0.0   0.0   2.0   0.0  ...   0.0   0.0   
Yogurt - Blueberry, 175 Gr          0.0   0.0   0.0   0.0  ...  25.0   0.0   
Yogurt - French Vanilla             2.0   0.0   0.0   1.0  ...   0.0   0.0   
Zucchini - Yellow                   0.0   0.0   0.0   0.0  ...   0.0   0.0   

                                                                             \
                                                                              
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube           0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart           0.0   0.0   0.0   0.0   0.0   0.0  25.0   
Appetizer - Sausage Rolls           0.0   0.0  25.0  25.0  25.0   0.0  25.0   
Apricots - Dried                    0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...                                 ...   ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                0.0  25.0   0.0   0.0   0.0   0.0   0.0   
Yoghurt Tubes                       0.0   0.0   0.0   0.0   0.0  25.0   0.0   
Yogurt - Blueberry, 175 Gr          0.0  25.0   0.0   0.0   0.0   0.0   0.0   
Yogurt - French Vanilla            25.0   0.0   0.0   0.0   0.0   0.0   0.0   
Zucchini - Yellow                   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                                         
                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube           0.0  
Appetizer - Mini Egg Roll, Shrimp   0.0  
Appetizer - Mushroom Tart           0.0  
Appetizer - Sausage Rolls           0.0  
Apricots - Dried                    0.0  
...                                 ...  
Yeast Dry - Fermipan                0.0  
Yoghurt Tubes                       0.0  
Yogurt - Blueberry, 175 Gr      

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
sim_matrix=squareform(pdist(pivot.T, metric='euclidean'))

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
distance = 1 / (1 + sim_matrix)
distances = pd.DataFrame(distance, index=pivot.columns, columns=pivot.columns)
distances.columns=sorted(list(set(data.CustomerID)))
distances.index=sorted(list(set(data.CustomerID)))
distances[33].sort_values(ascending=False)[1:6]

3317    0.087047
3535    0.087047
264     0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [10]:
top5 = list(distances[33].sort_values(ascending=False)[1:6].index)
productstop5=data2.loc[top5]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [13]:
top5Items=productstop5.reset_index()[['ProductName','Quantity']].groupby('ProductName').sum('Quantity').sort_values(by='Quantity',ascending=False)
top5Items

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [14]:
products=[i for i in top5Items.index if i not in data2.loc[33].index]
top5products=top5Items.loc[products].head(5).index.to_list()
top5products


['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [15]:
IDs=sorted(data['CustomerID'].unique().tolist())
recommendations=dict()
for i in IDs:
    top5 = list(distances[i].sort_values(ascending=False)[1:6].index)
    productstop5=data2.loc[top5]
    top5Items=productstop5.reset_index()[['ProductName','Quantity']].groupby('ProductName').sum('Quantity').sort_values(by='Quantity',ascending=False)
    products=[j for j in top5Items.index if j not in data2.loc[i].index]
    top5products=top5Items.loc[products].head(5).index.to_list()
    recommendations[str(i)]=top5products

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [16]:
pd.DataFrame(recommendations)

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,Soup - Campbells Bean Medley,Soupfoamcont12oz 112con,Butter - Unsalted,Olive - Spread Tapenade,Butter - Unsalted,Olive - Spread Tapenade,Cod - Black Whole Fillet,Bar - Granola Trail Mix Fruit Nut,"Veal - Inside, Choice",...,Cheese - Wine,Muffin Batt - Choc Chk,"Pork - Loin, Bone - In",Fenngreek Seed,"Placemat - Scallop, White","Soup - Campbells, Lentil",Skirt - 29 Foot,Chips Potato Salt Vinegar 43g,Crackers - Trio,Beans - Wax
1,Wine - Ej Gallo Sierra Valley,Muffin - Carrot Individual Wrap,Wine - Two Oceans Cabernet,"Veal - Inside, Choice",Sprouts - Baby Pea Tendrils,"Sauce - Gravy, Au Jus, Mix",Sprouts - Baby Pea Tendrils,Wine - Ej Gallo Sierra Valley,Wine - Magnotta - Belpaese,Sardines,...,Muffin - Carrot Individual Wrap,Papayas,Potatoes - Idaho 100 Count,"Wine - Red, Colio Cabernet",Rum - Mount Gay Eclipes,"Cheese - Brie,danish",Beans - Kidney White,Tea - English Breakfast,Pernod,Sherry - Dry
2,Soup - Campbells Bean Medley,Bay Leaf,Bread - Italian Roll With Herbs,"Beets - Candy Cane, Organic",Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Yoghurt Tubes,Appetizer - Mushroom Tart,Wine - Ej Gallo Sierra Valley,...,"Lentils - Red, Dry",Meldea Green Tea Liquor,Pants Custom Dry Clean,Grenadine,Tofu - Firm,Wiberg Super Cure,Milk - 1%,"Wine - Red, Harrow Estates, Cab",Tea - Jasmin Green,Pork - Hock And Feet Attached
3,Wine - Blue Nun Qualitatswein,Pork - Kidney,"Veal - Inside, Choice","Nut - Chestnuts, Whole","Pepper - Black, Whole","Peas - Pigeon, Dry","Veal - Inside, Choice","Pepper - Black, Whole","Peas - Pigeon, Dry","Pepper - Black, Whole",...,Meldea Green Tea Liquor,Wine - Chablis 2003 Champs,Cake - Box Window 10x10x2.5,Ice Cream Bar - Oreo Cone,Sobe - Tropical Energy,Pants Custom Dry Clean,Cheese - Taleggio D.o.p.,Bread - Raisin Walnut Oval,Pastry - Choclate Baked,Pears - Bosc
4,Chicken - Soup Base,Wanton Wrap,"Fish - Scallops, Cold Smoked",Lamb - Ground,Soup - Campbells Bean Medley,Bar - Granola Trail Mix Fruit Nut,"Pepper - Black, Whole",Soupfoamcont12oz 112con,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,...,Coffee - Dark Roast,Mussels - Frozen,Pie Filling - Cherry,Halibut - Fletches,"Cheese - Boursin, Garlic / Herbs",Shrimp - 31/40,Sprouts - Baby Pea Tendrils,Wine - Redchard Merritt,"Peas - Pigeon, Dry","Soup - Campbells, Cream Of"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [17]:
sim_matrix=squareform(pdist(pivot.T, metric='cosine'))
distance = 1 / (1 + sim_matrix)
distances = pd.DataFrame(distance, index=pivot.columns, columns=pivot.columns)
distances.columns=sorted(list(set(data.CustomerID)))
distances.index=sorted(list(set(data.CustomerID)))
recommendations=dict()
for i in IDs:
    top5 = list(distances[i].sort_values(ascending=False)[1:6].index)
    productstop5=data2.loc[top5]
    top5Items=productstop5.reset_index()[['ProductName','Quantity']].groupby('ProductName').sum('Quantity').sort_values(by='Quantity',ascending=False)
    products=[j for j in top5Items.index if j not in data2.loc[i].index]
    top5products=top5Items.loc[products].head(5).index.to_list()
    recommendations[str(i)]=top5products
pd.DataFrame(recommendations)

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Knife Plastic - White,Longos - Grilled Salmon With Bbq,Pickerel - Fillets,Bread - English Muffin,"Salsify, Organic",Wonton Wrappers,"Pasta - Penne, Rigate, Dry",Wine - Pinot Noir Latour,"Fish - Scallops, Cold Smoked",Cookies - Assorted,...,Vanilla Beans,Wine - Fume Blanc Fetzer,Spice - Peppercorn Melange,Sponge Cake Mix - Chocolate,Beef - Striploin Aa,"Spoon - Soup, Plastic",Sprouts - Baby Pea Tendrils,"Water, Tap","Peas - Pigeon, Dry",Cheese - Cheddarsliced
1,"Soup - Campbells, Beef Barley",Snapple Lemon Tea,"Water - Mineral, Natural",Olive - Spread Tapenade,Durian Fruit,Bread - Bistro White,Olive - Spread Tapenade,Crab - Imitation Flakes,Napkin White - Starched,Wine - Blue Nun Qualitatswein,...,Cheese - Cottage Cheese,Blackberries,French Pastry - Mini Chocolate,Bananas,Kellogs Special K Cereal,Beef - Montreal Smoked Brisket,Juice - Orange,Bananas,Squid U5 - Thailand,Wine - Toasted Head
2,Onions - Cippolini,General Purpose Trigger,Snapple - Iced Tea Peach,Bagel - Plain,Wine - Hardys Bankside Shiraz,Sausage - Breakfast,Spinach - Baby,Cheese - Parmesan Cubes,"Mushroom - Porcini, Dry",Pickerel - Fillets,...,Tomatoes Tear Drop,Coffee - Dark Roast,Scallop - St. Jaques,Initation Crab Meat,Pears - Bosc,Blackberries,Muffin Batt - Blueberry Passion,Wine - Redchard Merritt,Cheese - Taleggio D.o.p.,Cheese - Cottage Cheese
3,Tea - Herbal Sweet Dreams,"Thyme - Lemon, Fresh",Wine - Ej Gallo Sierra Valley,Pork - Hock And Feet Attached,Bread - Raisin Walnut Oval,"Tart Shells - Sweet, 4",Veal - Osso Bucco,Bouq All Italian - Primerba,Wine - Pinot Noir Latour,Beer - Rickards Red,...,Ice Cream Bar - Hageen Daz To,Hersey Shakes,"Beef - Tenderlion, Center Cut","Pork - Loin, Center Cut",Langers - Ruby Red Grapfruit,Cheese Cloth No 100,Beer - Original Organic Lager,Chocolate - Compound Coating,"Cheese - Brie, Triple Creme","Shrimp - Baby, Warm Water"
4,Muffin - Zero Transfat,Tomatoes Tear Drop,French Pastry - Mini Chocolate,Wiberg Super Cure,Gatorade - Xfactor Berry,"Juice - Cranberry, 341 Ml",Carbonated Water - Blackcherry,Whmis - Spray Bottle Trigger,Pork - Inside,"Hickory Smoke, Liquid",...,Table Cloth 120 Round White,Papayas,Water - Green Tea Refresher,"Sole - Dover, Whole, Fresh","Sauce - Gravy, Au Jus, Mix",Cake - Mini Cheesecake,"Nut - Chestnuts, Whole",Longos - Grilled Salmon With Bbq,Crackers - Trio,Jolt Cola - Electric Blue
